# Análisis y Limpieza de Datos -- Procedimiento para Catalogar Cáncer Luminal A y B

Este archivo contiene el proceso de limpieza, exploración y análisis de la base de datos del proyecto de de cáncer de mama en base a la información de los diagnosticos de diversos pacientes registrados por la Secretaria de Salud.

mas info sobre lo q buscamos


---

In [23]:
import pandas as pd

#Leemos la Base de Datos
df = pd.read_csv('./resources/2025_08_27 BD Proy Salud.csv', encoding='latin1')
pd.set_option('display.max_columns', None)

#hay dos Er_%, uno con _ y oyto con espacio, lo mismo con Pr_%, elimina los que tienen espacios
df = df.drop(columns=['ER %', 'PR %'])


#Renombramos columnas para agregar _ entre espacios

df.columns = df.columns.str.replace(' ', '_')
df.columns = df.columns.str.title()

#Sacamos columnas disponibles para facilidad de lectura posterior
unique_columns = df.columns.tolist()

print("Numero de columnas/variables:", len(unique_columns), "\n")
print("Columnas identificadas: \n", unique_columns)




Numero de columnas/variables: 109 

Columnas identificadas: 
 ['Id', 'Fecha_De_Registro', 'Fecha_De_Nacimiento', 'Fecha_De_Dx_Biopsia', 'Edad_Al_Dx_(Años)', 'Fecha_1°_Consulta_Om_Cecan', 'Peso_Al_Dx', 'Talla_Al_Dx', 'Imc', 'Edo_Menopausia_Dx', 'Preservación_De_La_Fertilidad', 'Prueba_Genetica', 'Resultado_Panel_Genético', 'Otra_Mutación', 'Significado', 'Cm_Bilateral', 'Cm_Asociado_Al_Embarazo', 'Ec', 'T', 'N', 'M', 'Tipo_Histológico', 'Otro', 'Grado', 'Er', 'Er_%', 'Pr', 'Pr_%', 'Her2_/_Neu', 'Her2_Ihq_+++', 'Fish/Sish', 'Ki67', 'Ki67_%', 'Ilv', 'Tils', 'Tils_%', 'Patología_Rev_Cecan', 'Tx_Neoadyuvante', 'Tx_Neoadyuvante_Cecan', 'Quimioterapia_Neoadyuvante', 'Esquema_Qt_Neoadyuvante', 'Antiher2_Neoadyuvante', 'Esquema_Antiher2_Neoadyuvante', 'Inmunoterapia_Neoadyuvante', 'Comentarios_Tx_Neoadyuvante', 'Respuesta_Patológica_Completa', 'Ypt', 'Ypn', 'Rbc', 'Fecha_De_Cx', 'Cx_Cecan', 'Tipo_Cx_Mama', 'Manejo_Axila', 'Requirio_2°_Cx_De_Mama', 'Cual', 'Rt_Adyuvante', 'Tipo_Rt_Ady', 'Analógo

En esta acción inicial: 
- Modificamos los nombres de las columnas para manipularlas de manera más sencilla
- Modificamos nombres similares para las variables de recurrencia de PR y ER para no ser confundidas con el porcentaje de PR y ER
- Retornamos una lista de todas las columnas para identificar mejoras, errores, y verificar cuales no son más utiles de una manera más eficiente


---
##### Creamos una sub-tabla con las columnas que corresponden a el objetivo de nuestro análisis

Seleccionamos datos de interés para el análisis de cáncer luminal, además de datos demográficos básicos


In [24]:
#Seleccionamos datos de interés para el análisis de cáncer luminal, además de datos demográficos básicos

luminal_columns = [
    'Id', 'Fecha_De_Registro', 'Fecha_De_Nacimiento', 'Fecha_De_Dx_Biopsia', 'Edad_Al_Dx_(Años)',
    'Fecha_1°_Consulta_Om_Cecan', 'Peso_Al_Dx', 'Talla_Al_Dx', 'Er_%', 'Pr_%', 'Her2_/_Neu',
    'Her2_Ihq_+++','Fish/Sish', 'Ki67_%', 'Grado'
]

#Nuevo DataFrame con columnas de interés
luminalDf = df[luminal_columns]

#Mostramos todas las columnas y los primeros registros
pd.set_option('display.max_columns', None)
luminalDf.head()



,Id,Fecha_De_Registro,Fecha_De_Nacimiento,Fecha_De_Dx_Biopsia,Edad_Al_Dx_(Años),Fecha_1°_Consulta_Om_Cecan,Peso_Al_Dx,Talla_Al_Dx,Er_%,Pr_%,Her2_/_Neu,Her2_Ihq_+++,Fish/Sish,Ki67_%,Grado
0,1003-24,1/5/2024,9/28/1977,11/24/2023,46,1/5/2024,76,1.64,10,100,Negativo,1+,No aplica,5,2
1,1008-18,1/25/2018,4/1/1961,1/12/2018,56,1/25/2018,102,1.61,10,5,Negativo,1+,No aplica,No aplica,3
2,1008-20,12/3/2019,2/10/1979,1/4/2020,40,1/17/2020,47,1.58,No aplica,No aplica,No aplica,No aplica,No aplica,No aplica,No aplica
3,1009-19,1/14/2019,1/23/1975,11/6/2018,43,1/14/2019,46,1.52,0,60,Negativo,1+,No aplica,20,1
4,1009-23,1/23/2023,11/30/1974,12/30/2022,48,1/25/2023,89,1.58,80,90,Negativo,0+,No aplica,No aplica,2


# Antes de comenzar...

Previo a cualquier análisis, necesitamos limpiar repeticiones, datos redundantes entre otros errores


In [3]:
#Observamos la cantidad de registros que tenemos

print("Cantidad de registros:", luminalDf.shape[0])


Cantidad de registros: 1366


In [4]:
#Limpiamos IDs repetidos y nulos, nos quedamos con los primeros repetidos

df = df.drop_duplicates(subset=['Id'], keep='first')
df = df[df['Id'].notna()]

#Mostramos la cantidad de registros después de la limpieza
print("Cantidad de registros después de la limpieza:", df.shape[0])

Cantidad de registros después de la limpieza: 1351


Al menos 15 registros estaban duplicados o nulos

---
##### Analizamos los tipos de datos de las columnas

In [5]:

pd.set_option('display.max_rows', None)

print("Tipos de datos de cada columna:")
print(luminalDf.dtypes)


Tipos de datos de cada columna:
Id                             object
Fecha_De_Registro              object
Fecha_De_Nacimiento            object
Fecha_De_Dx_Biopsia            object
Edad_Al_Dx_(Años)               int64
Fecha_1°_Consulta_Om_Cecan     object
Peso_Al_Dx                      int64
Talla_Al_Dx                   float64
Er_%                           object
Er_%                           object
Pr_%                           object
Pr_%                           object
Her2_/_Neu                     object
Her2_Ihq_+++                   object
Fish/Sish                      object
Ki67_%                         object
Grado                          object
dtype: object


##### Podemos identificar varios tipos de datos que no son correctos

- Podemos convertir las fechas a datetime.
- Los valores de Er_% y Pr_% deben ser convertidos a enteros.



In [37]:

#Cambiamos fechas a datetime
for col in luminalDf.columns:
    if 'Fecha' in col:
        luminalDf[col] = pd.to_datetime(luminalDf[col], errors='coerce') 

columnasFechas = [col for col in luminalDf.columns if 'Fecha' in col]


print(luminalDf[columnasFechas].dtypes)

Fecha_De_Registro             datetime64[ns]
Fecha_De_Nacimiento           datetime64[ns]
Fecha_De_Dx_Biopsia           datetime64[ns]
Fecha_1°_Consulta_Om_Cecan    datetime64[ns]
dtype: object


In [41]:

luminalDf = luminalDf.copy()  
luminalDf['Er_%'] = pd.to_numeric(luminalDf['Er_%'], errors='coerce').astype('Int64')
luminalDf['Pr_%'] = pd.to_numeric(luminalDf['Pr_%'], errors='coerce').astype('Int64')
luminalDf['Grado'] = pd.to_numeric(luminalDf['Grado'], errors='coerce').astype('Int64')
luminalDf['Ki67_%'] = pd.to_numeric(luminalDf['Ki67_%'], errors='coerce').astype('Int64')


print(luminalDf[['Er_%', 'Pr_%', 'Grado', 'Ki67_%']].dtypes)



Er_%      Int64
Pr_%      Int64
Grado     Int64
Ki67_%    Int64
dtype: object


---
Una vez que corregimos los tipos de datos, verificamos que si estén correctos


In [42]:
print(luminalDf.dtypes)

Id                                    object
Fecha_De_Registro             datetime64[ns]
Fecha_De_Nacimiento           datetime64[ns]
Fecha_De_Dx_Biopsia           datetime64[ns]
Edad_Al_Dx_(Años)                      int64
Fecha_1°_Consulta_Om_Cecan    datetime64[ns]
Peso_Al_Dx                             int64
Talla_Al_Dx                          float64
Er_%                                   Int64
Pr_%                                   Int64
Her2_/_Neu                            object
Her2_Ihq_+++                          object
Fish/Sish                             object
Ki67_%                                 Int64
Grado                                  Int64
dtype: object


##### Buscamos datos únicos para cada columna 

Identificamos datos únicos para observar irregularidades, duplicados, valores 

In [ ]:
#Obtenemos datos únicos para cada columna para estas columnas 'Er_%', 'Pr_%', 'Her2_/_Neu', que añada cada columna a un csv independiente

try:
    for col in ['Er_%', 'Pr_%', 'Her2_/_Neu', 'Her2_Ihq_+++','Fish/Sish', 'Ki67_%', 'Grado']:
        if col in luminalDf.columns:
            print(f'Columna: {col}\nValores únicos:\n{luminalDf[col].unique()}\n')
        else:
            print(f'Columna {col} no encontrada en luminalDf\n')
            
except Exception as e:
    pass


Columna: Er_%
Valores únicos:
<IntegerArray>
[  10, <NA>,    0,   80,  100,   15,   20,   90,    2,   85,   70,   12,   98,
   60,    5,   95,   30,   50,    4,   65,   40,    7,    1,   55,    8,    6,
   75,    3,   91,   25,   96,   35,   99,    9,   16]
Length: 35, dtype: Int64

Columna: Pr_%
Valores únicos:
<IntegerArray>
[ 100,    5, <NA>,   60,   90,    0,   95,   40,   80,    2,   20,   70,   10,
    3,   50,   25,   30,    8,    7,   35,   65,   75,   15,    1,   18,   22,
   12,    4,   85,   99,   98,    6,   88,   92,   68,    9]
Length: 36, dtype: Int64

Columna: Her2_/_Neu
Valores únicos:
['Negativo ' 'Negativo' 'No aplica' 'Positivo' 'Desconocido']

Columna: Her2_Ihq_+++
Valores únicos:
['1+' 'No aplica' '0+' '2+' '3+' 'Desconocido']

Columna: Fish/Sish
Valores únicos:
['No aplica' 'Negativo' 'Positivo' 'Desconocido']

Columna: Ki67_%
Valores únicos:
['5' 'No aplica' '20' '0' '90' '10' '50' '40' '60' '8' '65' '80' '95' '25'
 '30' '70' '15' '45' '3' '12' '47' '14' '7' '35

---
##### Hacemos la limpieza de los datos, esto incluyo:

- Reemplazar las filas numéricas con valores de "No Aplica" y "Desconocido" por NaN, con la justificación
  de que no cuentan con valores para catalogar en Luminal A y Luminal B

In [ ]:
#Borrar todas las filas que tengan en sus celdas de las columnas categóricas los valores 'No Aplica' o 'Desconocido'

import numpy as np
categorical_cols = ['Her2_/_Neu', 'Her2_Ihq_+++', 'Fish/Sish', 'Grado']  
# Reemplazar 'No Aplica' y 'Desconocido' por np.nan en las columnas categóricas


for col in categorical_cols:
    luminalDf[col] = luminalDf[col].replace(['No Aplica', 'No aplica', 'Desconocido', 'desconocido'], np.nan)


#Eliminar datos duplicados en las columnas categóricas, usar strip para verificar duplicidad

luminalDf['Her2_/_Neu'] = luminalDf['Her2_/_Neu'].str.strip()
luminalDf['Her2_Ihq_+++'] = luminalDf['Her2_Ihq_+++'].str.strip()
luminalDf['Fish/Sish'] = luminalDf['Fish/Sish'].str.strip()
luminalDf = luminalDf.drop_duplicates(subset=categorical_cols)

try:
    for col in ['Er_%', 'Pr_%', 'Her2_/_Neu', 'Her2_Ihq_+++','Fish/Sish', 'Ki67_%', 'Grado']:
        if col in luminalDf.columns:
            print(f'Columna: {col}\nValores únicos:\n{luminalDf[col].unique()}\n')

        else:
            print(f'Columna {col} no encontrada en luminalDf\n')
            
except Exception as e:
    pass

Columna: Er_%
Valores únicos:
<IntegerArray>
[  10, <NA>,    0,   80,  100,   20,   90,   15,   12,   70,   60,   50,    4,
   40,    7,   30,   55,    5,    6,   95,   85,    3,    8,   98,   99,    9]
Length: 26, dtype: Int64

Columna: Pr_%
Valores únicos:
<IntegerArray>
[ 100,    5, <NA>,   60,   90,    0,   95,   40,   80,    2,   20,   70,   10,
    3,   50,   25,   30,    8,    7,   35,   65,   75,   15,    1,   18,   22,
   12,    4,   85,   99,   98,    6,   88,   92,   68,    9]
Length: 36, dtype: Int64

Columna: Her2_/_Neu
Valores únicos:
['Negativo' nan 'Positivo']

Columna: Her2_Ihq_+++
Valores únicos:
['1+' nan '0+' '2+' '3+']

Columna: Fish/Sish
Valores únicos:
[nan 'Negativo' 'Positivo']

Columna: Ki67_%
Valores únicos:
<IntegerArray>
[   5, <NA>,   20,    0,   90,   40,   60,   95,   70,   10,   25,   30,   80,
    8,   15,    1,   12,   18,   35,    2,   67,    3,   45,   50]
Length: 24, dtype: Int64

Columna: Grado
Valores únicos:
<IntegerArray>
[ 100,    5, <NA>,   6

In [54]:
luminalDf.tail(15)

,Id,Fecha_De_Registro,Fecha_De_Nacimiento,Fecha_De_Dx_Biopsia,Edad_Al_Dx_(Años),Fecha_1°_Consulta_Om_Cecan,Peso_Al_Dx,Talla_Al_Dx,Er_%,Pr_%,Her2_/_Neu,Her2_Ihq_+++,Fish/Sish,Ki67_%,Grado
781,2071-22,2022-09-05,1976-07-09,2022-08-26,46,2022-09-05,82,1.54,8,6,Negativo,0+,NaN,10,6
798,2098-16,2015-04-21,1977-11-20,2015-08-01,37,2017-07-12,78,1.50,95,85,Negativo,0+,NaN,<NA>,85
832,2145-16,2016-08-12,1962-02-02,2016-09-01,54,2017-05-08,85,1.59,98,88,Negativo,0+,NaN,15,88
846,2164-24,2024-06-20,1951-05-16,2024-06-13,73,2024-07-17,57,1.54,100,1,Positivo,2+,Positivo,<NA>,1
848,2166-24,2024-06-25,1948-11-05,2023-07-25,74,2024-07-18,69,1.57,99,92,Negativo,0+,NaN,45,92
880,2214-16,2015-05-11,1967-09-18,2012-01-01,44,2017-06-26,98,1.60,<NA>,<NA>,Negativo,0+,NaN,<NA>,<NA>
904,2262-22,2022-10-20,1973-03-05,2022-09-29,49,2022-10-20,70,1.54,0,3,Positivo,3+,NaN,<NA>,3
1046,2527-16,2016-10-31,1953-07-22,2017-07-01,63,2017-02-14,60,1.60,90,68,Negativo,1+,NaN,50,68
1055,2566-19,2017-08-31,1978-02-22,2019-12-01,41,2019-12-16,64,1.53,100,60,Negativo,2+,Negativo,<NA>,60
1072,2626-18,2018-05-15,1982-04-19,2018-09-01,36,2018-10-08,74,1.55,30,25,Positivo,3+,NaN,<NA>,25
